In [29]:
import pandas as pd
import numpy as np
import plotly.express as go
import glob
import folium
from folium.plugins import MarkerCluster
from folium import IFrame
from folium import plugins
from sklearn.cluster import KMeans
from datetime import datetime
import os
import matplotlib.pyplot as plt
today=datetime.today().strftime('%Y-%m-%d')


In [30]:
df = pd.read_csv("Traffic_Accident_Locations.csv")


In [31]:
df = df.drop(columns=["AccidentID","County","AgencyCode","City","AddressOrStreetName",
                "geox","geoy","CaseNumber","RoadType","RoadCharacter","SupplementalReportTaken","PDUnit",
                "PDDivision","PDShift","PDDistrict","PrivateProperty"], axis=1)

In [32]:
df[["Day","Time"]] = df["DateTimeOccurred"].str.split(" ",expand=True)

In [33]:
df = df[["OBJECTID",'Day', 'TimeOccurred','X', 'Y','DayOfWeek','UnitsInvolved', 'RoadLanes', 'RoadSurface', 'RoadDefects','LocationOfImpact', 'MannerOfCollision', 'LightingCondition',
        'WeatherCondition', 'TrafficFlow', 'PDZone']]

In [34]:
pd.DatetimeIndex(df['Day']).year.unique()

Int64Index([2016, 2019, 2018, 2017, 2015, 2014, 2013, 2012, 2011, 2010, 2020,
            2021],
           dtype='int64', name='Day')

Since I don`t have missing coordinates, it`s possible to generate thte PDZone Using KNN Technique by getting the nearest coordinate and imput the same PDZone

** Haversine’s Equation **

In [35]:
df_missing_PD = df[df["PDZone"].isna()]

In [36]:
df_PD_ok = df[~df["PDZone"].isna()]

In [37]:
from math import radians, cos, sin, asin, sqrt
def dist(lat1, long1, lat2, long2):
    """
Replicating the same formula as mentioned in Wiki
    """
    # convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [38]:
def find_nearest(lat, long):
    distances = df_PD_ok.apply(
        lambda row: dist(lat, long, row['X'], row['Y']), 
        axis=1)
    return df_PD_ok.loc[distances.idxmin(), 'OBJECTID']

In [39]:
if len(glob.glob("./complete_df.csv"))> 0:
    df_completed = pd.read_csv("complete_df.csv")
else:
    df_missing_PD['name'] = df_missing_PD.apply(
        lambda row: find_nearest(row['X'], row['Y']), 
        axis=1)
    # To check the data frame if it has a new column of hotel name (for each and every member's location in the list)
    reference = df_PD_ok[["OBJECTID",'PDZone']]
    df_missing_PD = df_missing_PD.merge(reference,
        left_on='name', 
        right_on='OBJECTID')
    df_missing_PD = df_missing_PD[['OBJECTID_x', 'Day', 'TimeOccurred', 'X', 'Y', 'DayOfWeek',
        'UnitsInvolved', 'RoadLanes', 'RoadSurface', 'RoadDefects',
        'LocationOfImpact', 'MannerOfCollision', 'LightingCondition',
        'WeatherCondition', 'TrafficFlow', 'PDZone_y']]
    df_missing_PD = df_missing_PD.rename(columns={"OBJECTID_x":"OBJECTID","PDZone_y":"PDZone"})
    df_completed = df_missing_PD.append(df_PD_ok)
    df_completed.to_csv("./complete_df.csv")

In [40]:
df_completed = df_completed.set_index("Day")
df_completed.index = pd.to_datetime(df_completed.index)

In [41]:
df_completed['TimeOccurred'] = pd.to_datetime(df_completed['TimeOccurred'], errors='coerce')
df_completed['period'] = (df_completed['TimeOccurred'].dt.hour % 24 + 4) // 4
df_completed['period'].replace({1: 'Late Night',
                      2: 'Early Morning',
                      3: 'Morning',
                      4: 'Noon',
                      5: 'Evening',
                      6: 'Night'}, inplace=True)

In [42]:
df_completed['year'] = pd.DatetimeIndex(df_completed.index).year

In [43]:
df_completed["TimeOccurred"] = pd.to_datetime(df_completed['TimeOccurred']).dt.time

In [44]:
df_count_accidents = df_completed.groupby(["Day"])["DayOfWeek"].count().reset_index().sort_values(by='DayOfWeek', ascending=False)

In [45]:
df_count_accidents = df_count_accidents.set_index("Day")
df_count_accidents.index = pd.to_datetime(df_count_accidents.index)


In [46]:
df_count_accidents=df_count_accidents.rename(columns={"DayOfWeek":"Count"}).sort_index()

In [47]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
cal = calendar()
holidays = cal.holidays(start=df_count_accidents.index.min(), end=df_count_accidents.index.max())

df_count_accidents['Holiday'] = df_count_accidents.index.isin(holidays)


In [48]:
df_count_accidents = df_count_accidents[["Holiday","Count"]]

Do we have more accidents on holiday?

Based on our data, it indicates that in Johns Creek it doesn`t!

In [49]:
df_completed.groupby(["DayOfWeek", "year"])["OBJECTID"].count().reset_index(name='count').sort_values(['year','count'], ascending=[False,False]).head(7)

,DayOfWeek,year,count
83,Wednesday,2021,208
59,Thursday,2021,199
71,Tuesday,2021,128
11,Friday,2021,126
23,Monday,2021,117
47,Sunday,2021,77
35,Saturday,2021,55


At least from my expectation both informations are quite counterintuitive. Weekends are also not the days with most accidents!

Create Data Visualization to see the accidents distribution per year.
Saving in different html per year

In [51]:
years = df_completed["year"].unique()

In [52]:
weird_lat_long = []
def marker_tomap(x,y,index):
    ## It was necessary to create this -90 filter since some data seemed to be posted incorrectly reporting accidents in the middle of the coast of California.
    if y < -90:
        weird_lat_long.append([index,x,y])
        return 0

    
    if row.RoadSurface == "Dry":
        colors_ = "orange"
        folium.Marker(location=[x,y],\
        icon = folium.Icon(color=colors_,icon="glyphicon glyphicon-leaf"),\
        popup = "Latitude:" + str(x) + "\nLongitude:" + str(y) + "\nTimeOccurred:" + str(row.TimeOccurred) +"  \n  "+ "\nUnitsInvolved:\n"+str(row.UnitsInvolved) + "\nPeriod:" + str(row.period)).add_to(m)
        
    elif row.RoadSurface == "Wet":
        colors_ = "blue"
        folium.Marker(location=[x,y],\
        icon = folium.Icon(color=colors_,icon='glyphicon glyphicon-tint'),
        popup = "Latitude:" + str(x) + "\nLongitude:" + str(y) + "\nTimeOccurred:" + str(row.TimeOccurred) +"  \n  "+ "\nUnitsInvolved:\n"+str(row.UnitsInvolved) + "\nPeriod:" + str(row.period)).add_to(m)

    else:
        colors_="black"
        folium.Marker(location=[x,y],\
        icon = folium.Icon(color=colors_),
        popup = "Latitude:" + str(x) + "\nLongitude:" + str(y) + "\nTimeOccurred:" + str(row.TimeOccurred) +"  \n  "+ "\nUnitsInvolved:\n"+str(row.UnitsInvolved) + "\nPeriod:" + str(row.period)).add_to(m)


In [53]:

for year in years:
    m = folium.Map([33.8, -84.24], tiles='Cartodb positron', zoom_start=10)
    minimap = plugins.MiniMap(toggle_display=True)
    for (index, row) in df_completed.iterrows():
        if row.year == year:
            marker_tomap(row.Y,row.X,index)

    m.add_child(minimap)
    m.add_child(folium.LatLngPopup())
    plugins.ScrollZoomToggler().add_to(m)
    plugins.Fullscreen(position='topleft').add_to(m)
    m.save(os.path.join('results', str(year)+'result.html'))
    exported_file1 = str(year)+'-result.html'